# The Age of A star is detirmined by its Mass, B-V, and Luminosity, and this is the new target of my data.
# Also as a sperarate task, we will discover wether if any of our stars may turn into a blackhole or not.


<img src="https://www.schoolsobservatory.org/sites/default/files/astro/starcycle.jpg">

# This data has some star feature like:
- Visual Apparent Magnitude of the Star (Vmag)
- Distance Between the Star and the Earth (Plx)
- Standard Error of Plx (e_Plx)
- B-V color index (B-V)
- Spectral type (SpType)
- Absolute Magnitude of the Star (Amag)
# The real target which was published for this data was to spectral classification:

<img src="https://astronomy.com/-/media/Images/Magazine%20Articles/2021/04/ScreenShot20210420at3.19.16PM.jpg?mw=600">



# here are some info about the features: 
- A hot star has a B-V color index close to 0 or negative, while a cool star has a B-V color index close to 2.0. Other stars are somewhere in between.
- Cold  Star means its age is older

- Dwarf (0) and Giant (1)
- The more luminous an object, the smaller the numerical value of its absolute magnitude

# To Understand the Relation between the feature, I will use a finely distrebuted data to understand the basic features, the I'll use the real dataset.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import plotly.express as px
import plotly.graph_objects as go
from dataprep.datasets import load_dataset
from dataprep.eda import plot
from dataprep.eda import plot_diff
from dataprep.eda import plot_correlation
from dataprep.eda import plot_missing
import math

# 1) The Example dataset:

In [ ]:
df= pd.read_csv("//Users//omniaelmenshawy//Desktop///Star.csv")

In [ ]:
df.head(5)

In [ ]:
df.describe().T.style.background_gradient(subset=['mean','std','50%','count'], cmap='PuBu')

In [ ]:
df.corr()

## Visualizing eah star by its name:

In [ ]:
df['Type Names']= df['Star type']

In [ ]:
df['Type Names'].replace(0  ,'Brown Dwarf',inplace=True)
df['Type Names'].replace(1  ,'Red Dwarf',inplace=True)
df['Type Names'].replace(2  ,'White Dwarf',inplace=True)
df['Type Names'].replace(3  ,'Main Sequence',inplace=True)
df['Type Names'].replace(4  ,'Supergiant',inplace=True)
df['Type Names'].replace(5  ,'Hypergiant',inplace=True)

## label Encoding for better Visualization:

In [ ]:
df['color']= df['Star color']

In [ ]:
df['Spectral']= df['Spectral Class']

In [ ]:
from sklearn.preprocessing import LabelEncoder
color_le = LabelEncoder()
df['color'] = color_le.fit_transform(df.color)

In [ ]:
color_le.classes_

In [ ]:
Spectral_le = LabelEncoder()
df['Spectral'] = Spectral_le.fit_transform(df.Spectral)

In [ ]:
Spectral_le.classes_

### Now we have:
#### spectral  B = 1 , A = 0, O = 6, M= 5, K=4 , G= 3
#### color Red = 10, Blue = 0, Blue white = 2

In [ ]:
df.columns

## Radius and Star type Relation:
####  From the Below Observation we understand that Radius of each star typr has a huge difference in magnitude, for example, Hyper Giants has the biggest radius, which means that they are the biggest in mass also, while the dwarfs are the smallest stars existed.


### Taking the radius as the size to detirmine the real difference in size:

- Now we can observe the real difference.
- The star gets larger, redder, and more luminous as it expands and cools

In [ ]:
fig = px.scatter(df,
                 x="Star type", y='Radius(R/Ro)', size="Radius(R/Ro)", color="Type Names",
                  size_max=60,
                 template='plotly_dark', title='Star Type vs radius as size')
fig.show()

## Spectral Class and its Tempreature:

- we can clearly notice that classes which starts with M, K, and G are the lowest in Tempreature, and these are the classes which we are interested in as we are looking for old stars, as they are more likly to die soon.

In [ ]:
fig = px.bar(df, x = 'Spectral Class',color="Temperature (K)",hover_name="Type Names", template = 'plotly_dark', title='Spectral Type vs temperature')
fig.show()

## Hertzsprung-Russell Diagram: 
- In the Hertzprung-Russell diagram the temperatures of stars are plotted against their luminosities. The position of a star in the diagram provides information about its present stage and its mass. Stars that burn hydrogen into helium lie on the diagonal branch, the so-called main sequence.
<img src="http://cdn.eso.org/images/screen/eso0728c.jpg">

## Luminosity vs. Surface Temperature in degrees to visualize the Hertzsprung-Russell Diagram:

- Our data has a similar shape as the diagram as the supergiants and hypergiants ar in the top of the graph.
- Other normal starsspends their lives in the main sequence.
- Most of the dwarfsare in the bottom of the graphwith the lowest temperatures and luminosity.

In [ ]:
fig = px.scatter(df, x="Temperature (K)", y="Luminosity(L/Lo)", hover_name="Type Names",
                 color="Type Names",log_x=True, log_y=True, template = 'plotly_dark', title='Luminosity vs. Surface Temperature ')
fig.show()

## Final conclusion from the example data:
- we need the same features in the new data to deteirmine the location of each star in the HR diagram

# The Real dataset:

In [ ]:
path = "//Users//omniaelmenshawy//Desktop///Star39552_balanced.csv"
df = pd.read_csv(path)
df.head(5)

In [ ]:
df.describe().T

## Converting Columns data type to float values

In [ ]:
df["Vmag"] = pd.to_numeric(df["Vmag"], downcast="float", errors='coerce')
df["Plx"] = pd.to_numeric(df["Plx"], downcast="float", errors='coerce')
df["e_Plx"] = pd.to_numeric(df["e_Plx"], downcast="float", errors='coerce')
df["B-V"] = pd.to_numeric(df["B-V"], downcast="float", errors='coerce')

## Data Report and feature visualization:

In [ ]:
from dataprep.eda import create_report
report = create_report(df, title='My Report')

In [ ]:
report

## Reducing the error:

In [ ]:
threshold = df['e_Plx'].mean() + 0.5

df = df[df['e_Plx'] < threshold ]

In [ ]:
error = df['e_Plx']

In [ ]:
report = create_report(error, title='My Report')

In [ ]:
report

## Changing the target to a normal feature as it is not our intereset now:

In [ ]:
df['GiantOrDwarf'] = df['TargetClass']

## From kelvins to degrees:
- C = K - 273.15

In [ ]:
#df['Temp']= df['Temp']- 273.15

## Calculating the Luminosity as L/Lo:

- Absolute magnitude and luminosity are related with the formula

### M = -2.5 * log₁₀(L / L₀)

### where:

- M is the absolute magnitude of the star;
- L is its luminosity;
- L₀ is the zero-point luminosity, equal to 3.0128 * 10²⁸ W.

In [ ]:
df['Amag(arcsec)'] = df['Amag']*0.001

In [ ]:
df['Amag(parsec)'] = df['Amag(arcsec)']*0.25

a star with a mass greater than 20 times the mass of our Sun may ultimately become a black hole;
V(orbit) = SQRT(GM/R) 

In [ ]:
df.describe().T

# Is the Star Giant or Dwarf?

In [ ]:
df['GiantOrDwarf'].value_counts()

In [ ]:
colors_blue = ["#132C33", "#264D58", '#17869E', '#51C4D3', '#B4DBE9']
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']

In [ ]:
df.shape

In [ ]:
df.columns

# Trying to understand Some feature Relations

In [ ]:
fig = px.scatter(df, x="Temp", y="Amag",
                 color="B-V",log_x=True, log_y=True, template = 'plotly_dark', title= ' Amag Vs. Temp')
fig.show()

# I need my stars to be in the main sequence to be able to calculate their Mass correctly as: 
<img src="http://www.astronomy.ohio-state.edu/~pogge/Ast162/Unit2/Images/MLrel_eqn.gif">


- Main sequence stars have a Morgan-Keenan luminosity class labelled V

In [ ]:
def Main_Seq(row):
    if row['GiantOrDwarf'] == 1 :
        for i in range(len(df['SpType'])):
            if "V" in df.loc[i,'SpType']:
                return 1
               
    else:
        return 0
    

In [ ]:
df['LifeTime'].value_counts()

In [ ]:
df.columns

In [ ]:
related = df[['Vmag','SpType', 'Luminosity', 'B-V', 'Mass','MainSeq', 'LifeTime']]

In [ ]:
fig = px.scatter_matrix(related, color = 'LifeTime')
fig.show()

In [ ]:
related = df[['SpType', 'LifeTime']]
fig = px.scatter_matrix(related, color = 'LifeTime')
fig.show()

In [ ]:
related = df[['MainSeq', 'LifeTime']]
fig = px.scatter_matrix(related, color = 'LifeTime')
fig.show()

In [ ]:
df.columns

In [ ]:
df['SpType'].value_counts()

In [ ]:
report = create_report(df, title='My Report')

In [ ]:
report

In [ ]:
pl.figure(figsize=(10,6))
ax = sns.scatterplot(x='B-V',y='LifeTime',data=df,palette='magma',hue='MainSeq')
ax.set_title('Scatter plot of LifeTime and B-V')

sns.lmplot(x="SpType", y="LifeTime", hue="MainSeq", data=df, palette = 'magma', size = 8)

In [ ]:
df.head()

In [ ]:
y = df['LifeTime']

# Select Featurestrai
x= df[['B-V','Amag','Luminosity','GiantOrDwarf','MainSeq', 'Mass','E' ]]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [ ]:
lr = LinearRegression()

knn = KNeighborsRegressor(n_neighbors=10)

dt = DecisionTreeRegressor(max_depth = 3)

rf = RandomForestRegressor(max_depth = 3, n_estimators=500)

ada = AdaBoostRegressor( n_estimators=50, learning_rate =.01)

gbr = GradientBoostingRegressor(max_depth=2, n_estimators=100, learning_rate =.2)

xgb = XGBRegressor(max_depth = 3, n_estimators=50, learning_rate =.2)

cb = CatBoostRegressor(learning_rate =.01, max_depth =5, verbose = 0)

regressors = [('Linear Regression', lr), ('K Nearest Neighbours', knn),
               ('Decision Tree', dt), ('Random Forest', rf), ('AdaBoost', ada),
              ('Gradient Boosting Regressor', gbr), ('XGBoost', xgb), ('catboost', cb)]

In [ ]:
from sklearn import metrics

In [ ]:
predictions = lr.predict(x_test)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
l = mean_absolute_percentage_error(y_test, predictions)
print('Mean Absolute Percentage Error is:', l )
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
sns.distplot((y_test-predictions),bins=20);

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
predictions = gbr.predict(x_test)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
l = mean_absolute_percentage_error(y_test, predictions)
print('Mean Absolute Percentage Error is:', l )
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
sns.distplot((y_test-predictions),bins=20);

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
l = mean_absolute_percentage_error(y_test, predictions)
print(l)

In [ ]:
sns.distplot((y_test-predictions),bins=20);

In [ ]:
predictions = knn.predict(x_test)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
l = mean_absolute_percentage_error(y_test, predictions)
print('Mean Absolute Percentage Error is:', l )
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
sns.distplot((y_test-predictions),bins=20);

In [ ]:
predictions = dt.predict(x_test)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
l = mean_absolute_percentage_error(y_test, predictions)
print(l)

In [ ]:
sns.distplot((y_test-predictions),bins=20);

In [ ]:
predictions = rf.predict(x_test)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
l = mean_absolute_percentage_error(y_test, predictions)
print('Mean Absolute Percentage Error is:', l )
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
sns.distplot((y_test-predictions),bins=20);

In [ ]:
from sklearn.metrics import mean_absolute_error


mape = mean_absolute_error(y_test, predictions)*100
print(mape)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
l = mean_absolute_percentage_error(y_test, predictions)
print(l)

In [ ]:
predictions = ada.predict(x_test)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
l = mean_absolute_percentage_error(y_test, predictions)
print('Mean Absolute Percentage Error is:', l )
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
sns.distplot((y_test-predictions),bins=20);

# Scientific resources:


- Coursera Astronomy Data Driven Course From The University of Sydeny: 
#### https://www.coursera.org/learn/data-driven-astronomy/home/welcome


- Hertzsprung-Russell Diagram:
#### https://astronomy.swin.edu.au/cosmos/h/hertzsprung-russell+diagram


- Star Mass:
#### https://www.researchgate.net/figure/Mass-radius-relation-for-Kepler-red-giants-with-RGB-stars-in-blue-and-clump-stars-in_fig6_257882348


- Black Hole propertries: 
#### https://public.nrao.edu/ask/properties-of-stars-which-result-in-black-holes/
#### https://public.nrao.edu/ask/what-is-the-critical-mass-at-which-a-star-becomes-a-black-hole/


- PRINCIPLES OF STELLAR STRUCTURE (OHIO State University)
#### http://www.astronomy.ohio-state.edu/~dhw/Intro/lec6.html
#### http://www.astronomy.ohio-state.edu/~pogge/Ast162/Unit2/structure.html


- Finding Our Sun From Amag: 
#### https://arxiv.org/abs/1804.07788 


- Blackbody Radiation and Quantization of Energy fot Luminosity and B-V:
#### https://web.njit.edu/~gary/321/Lecture2.html


- Finding Temperature and Spectral Type Based on Ballesteros equation: 
#### http://astro.physics.uiowa.edu/ITU/labs/professional-labs/photometry-of-a-globular/part-2-finding-temperature.html


- Calculating the Radius of a Star:
#### https://cas.sdss.org/dr4/en/proj/advanced/hr/radius1.asp


- Brightest Stars: Luminosity & Magnitude:
#### https://www.space.com/21640-star-luminosity-and-magnitude.html


- Distance Parallax equation:
#### http://astronomyonline.org/Science/Parallax.asp 




In [ ]:
df

In [ ]:
df.to_csv('Detection2.csv', index=False)